## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
dims = (1,15,15)
n_outputs = 10

np.random.seed(seed=39)
weights = (np.random.rand(n_outputs,np.product(dims)) - 0.5) / 4
biases = (np.random.rand(n_outputs) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims),
    layers.Dense(weights=weights, biases=biases),
    layers.MonitorLayer(),
])

input0 = quartz.probe(loihi_model.layers[0].blocks[-1])
relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks[::3]]
calcs = [quartz.probe(block.neurons[0]) for block in loihi_model.layers[1].blocks[::3]]

values = np.random.rand(1,*dims) # np.ones((np.product(dims))) * 0.5 np.zeros((np.product(dims))) # 
quantized_values = (values*t_max).round()/t_max
quantized_biases = (biases*t_max).round()/t_max

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims), out_features=n_outputs), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor((quantized_biases)))
model_output = pt_model(torch.tensor(quantized_values).flatten()).detach().numpy()
model_output

In [ ]:
loihi_model(values, t_max)#, steps_per_image=2500)

In [ ]:
ok = [relco.plot() for relco in relcos]

In [ ]:
ok = [calc.plot() for calc in calcs]

In [ ]:
dim_input = (1,10,1,)
dim_output = 1
t_max = 2**8
np.random.seed(seed=47)
weights = np.ones((dim_output,np.product(dim_input))) / 255

loihi_model = quartz.Network([
    layers.InputLayer(dims=dim_input),
    layers.Dense(weights=weights, biases=None),
    layers.MonitorLayer(),
])
values = np.ones((1, *dim_input))
quantized_values = (values*t_max).round()/t_max

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dim_input), out_features=dim_output), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor(np.zeros(dim_output)))
model_output = pt_model(torch.tensor(quantized_values).flatten()).detach().numpy()
model_output

In [ ]:
loihi_output = loihi_model(values, t_max)
loihi_output

In [ ]:
round(0.03921569 * t_max) / t_max

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (2,5,5)
weight_dims = (1,2,2,2)

kernel_size = weight_dims[2:]
np.random.seed(seed=27)
weights = (np.random.rand(*weight_dims)-0.5) / 2 # np.ones((weight_dims)) # 
biases = (np.random.rand(weight_dims[0])-0.5) # np.zeros((weight_dims[0])) # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=None),
    layers.MonitorLayer(),
])

input0 = quartz.probe(loihi_model.layers[0].blocks[0])
relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks]

values = np.random.rand(2, *input_dims) / 2 # np.zeros((np.product(input_dims)))

quantized_values = (values*t_max).round()/t_max
#quantized_values = quantized_values.reshape(*input_dims)

model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=kernel_size), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(weights))
model[0].bias = torch.nn.Parameter(torch.tensor(np.zeros_like(biases)))
model_output = model(torch.tensor(quantized_values.reshape(2, *input_dims))).detach().numpy()
model_output

In [ ]:
values.shape

In [ ]:
probe = loihi_model(values, t_max, steps_per_image=2500)

In [ ]:
probe

In [ ]:
probe.output()

In [ ]:
ok = [relco.plot() for relco in relcos]

## Max Pooling layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (1,10,10)
kernel_size = [2,2]

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.MaxPool2D(kernel_size=kernel_size),
    layers.MonitorLayer(),
])

pool0 = quartz.probe(loihi_model.layers[1].blocks[0])

np.random.seed(seed=45)
values = np.random.rand(np.product(input_dims))
values[values.shape[0]//2:] = 0
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max

model = nn.Sequential(nn.MaxPool2d(kernel_size=kernel_size, stride=kernel_size[0]), nn.ReLU())
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
values

## Conv Maxpool layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = ( 2,7,7)
weight_dims = ( 4,2,5,5)

input_dims = ( 1,10,10)
weight_dims = ( 6,1,3,3)
t_max = 2**9
weight_acc = 2**7
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=46)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 4 # np.zeros(weight_dims) #
biases = np.zeros(weight_dims[0]) # (np.random.rand(weight_dims[0])-0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights, biases=biases, pool_kernel_size=pooling_kernel_size, weight_acc=weight_acc),
    layers.MonitorLayer(),
])
loihi_model1 = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.Conv2D(weights=weights, biases=biases, weight_acc=weight_acc),
    layers.MaxPool2D(kernel_size=pooling_kernel_size, stride=pooling_stride, weight_acc=weight_acc),
    layers.MonitorLayer(),
])

trigg0 = quartz.probe(loihi_model.layers[1].blocks[0])
# conv00_block = quartz.probe(loihi_model.layers[1].blocks[7])
# conv00 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[0])
# conv01 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[1])
# conv02 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[2])
# conv03 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[3])
# conv04 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[4])
# conv05 = quartz.probe(loihi_model.layers[1].blocks[7].neurons[5])
conv22 = quartz.probe(loihi_model.layers[1].blocks[-1])

values = np.random.rand(np.product(input_dims))
quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(weights))
model[0].bias = nn.Parameter(torch.tensor(biases))
model_output = model(torch.tensor(values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()
model_output

In [ ]:
loihi_model(values, t_max).reshape(model_output.shape)

In [ ]:
loihi_model1(values, t_max).reshape(model_output.shape)

In [ ]:
loihi_model.layers[1].blocks[0].neurons[0].outgoing_synapses()

In [ ]:
loihi_model.layers[1].blocks[-1].second()

In [ ]:
loihi_model1.layers[2].output_blocks()[0].output_neurons()[1].incoming_synapses()